In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from setdir import *

In [3]:
# Slocum gliders
unit_list = ['unit_398', 'unit_409']

# File time stamp
yyyymmdd = '20220105'
yyyymmdd = datetime.datetime.now().strftime('%Y%m%d')

# Time limits
datestart = '2021-12-12'
#tstart = time.mktime(datetime.datetime.strptime(datestart,'%Y-%m-%d').timetuple())
tstart = pd.Timestamp('2021-12-12T00')

# Initialise the figure directory
figdir = create_figdir()


font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}

plt.rc('font', **font)

In [4]:
# Add a time column, add a pressure_dbar column, and trim the start (pre-deployment)

if 0:
    axes = plt.subplots(nrows=3, ncols=1,figsize=(20,10))

count = 0
# Process a pickle
for i in unit_list:
    count += 1
    infile = i+'_'+yyyymmdd+'.pkl'
    infile_with_path = cat_interim_path(infile)
    data_df = pd.read_pickle(infile_with_path)

    data_df['time'] = data_df.timestamp.apply(lambda x: datetime.datetime.fromtimestamp(x*0.001))

    # Cut data to post deployment
    data_file_2021 = data_df[data_df.time>=tstart].copy()
    
    # Change pressure from bars to dbars
    data_file_2021['pressure_dbar'] =  data_file_2021.sci_water_pressure * 10

    
    # Remove negative salinities
    df1 = data_file_2021['derived_salinity']
    df2 = df1.where(df1>0)
    data_file_2021['derived_salinity'] = df2

    if 0:
        # Plot pressure against time
        ax1 = plt.subplot(2,2,2*count-1)
        data_file_2021.plot(x='time', y='pressure_dbar', ylabel='pressure (dbar)', title=i, ax=ax1)
        #    if count==1:
        #        ax1.xaxis.set_ticklabels([])
        

        ax1.invert_yaxis()
    
        # Plot the positions?
        ax3 = plt.subplot(1,2,2)
        data_file_2021.plot(x='m_gps_lon',y='m_gps_lat', ax=ax3)
        
    # Save an updated pickle
    outfile = i+'_'+yyyymmdd+'_edit.pkl'
    outfile_with_path = cat_interim_path(outfile)
    
    data_file_2021.to_pickle(outfile_with_path)



In [5]:
data_file_2021.head(3)

,timestamp,sci_water_pressure,sci_water_temp,sci_water_cond,sci_oxy4_oxygen,derived_salinity,derived_potential_density,derived_potential_temperature,m_gps_lon,m_gps_lat,time,pressure_dbar
19927,1639267200000,15.171500,2.52525,3.016935,401.154999,33.591046,1026.801732,2.516804,NaN,NaN,2021-12-12 00:00:00,151.714999
19928,1639267260000,14.148500,2.33185,2.992350,403.725998,33.496583,1026.741939,2.324236,NaN,NaN,2021-12-12 00:01:00,141.485000
19929,1639267320000,13.178333,2.17725,2.972600,405.893005,33.419560,1026.692443,2.170355,NaN,NaN,2021-12-12 00:02:00,131.783334
